In [ ]:
# neo4j 사용법

# create node
create(:person{name:'kim',age:22})
create(:person{name:'park',age:26})

# create relationship
# kim node 와 park node가 존재해도 match를 이용하지 않으면 새로운 노드를 생성하게 됨
create(:person{name:'kim'})-[:knows{since:2002}]->(:person{name:'park'})
# match 사용
match(p1:person{name:'kim'}) 
match(p2:person{name:'park'}) 
create(p1)-[:knows{since:2002}]->(p2)

# Delete Node
# delete single node 
match(n:person{name:'UNKNOWN'})
delete n
# delete all nodes nad relationships
match(n)
detach delete n
# delete a node with all its relationships
match(n{name:'andy'})
detach delete n

# delete relationship only
match (n{name:'andy'})-[r:KNOWS]->()
delete r

#Querying Graphs

match (a:Person{name:'jim'})-[:knows]->(b)-[:knows]->(c),(a)-[:KNOWS]->(c)
return b,c
# == 
match (a:Person)-[:KNOWS]->(b)-[:KNOWS]->(c),(a)-[:KNOWS]->(c)
where a.name='jim'
return b,c

In [1]:
#Dijkstra 예제(1/4)
#노드와 관계 생성
from py2neo import Graph, Node, Relationship
import heapq

g = Graph()
n = 8
#a = [from, to, weight]
a =  [[0,1,11],
     [0,2,9],
     [0,3,8],
     [1,4,8],
     [1,5,8],
     [2,1,3],
     [2,3,6],
     [2,6,1],
     [3,6,10],
     [4,5,7],
     [5,2,12],
     [5,7,5],
     [6,7,2],
     [7,4,4]]
nodes = [Node("Node",name=str(i)) for i in range(n)]  # 0-7의 이름을 가진 노드 생성

#transaction 실행
tx = g.begin()
for i in a:
    re = Relationship(nodes[i[0]],"DIST",nodes[i[1]],dist = str(i[2])) # 관계 생성 ( node, 간선, node, property)
    tx.create(re)
tx.commit()

#transaction 종료 검사
if tx.finished()==False:
    print('transaction is still open')
else:
    print('transaction has been completed')

transaction has been completed


In [2]:
# Dijkstra 예제(2/4)
# DB 데이터 불러오기 (dataframe to dictionary)
# node의 갯수가 많아지면 안되고 
df_to_dict = []

# 0번 노드부터 관계 값을 가져와 dataframe으로 저장
for i in range(n):
    df = g.run("match(a)-[r]->(b)"
              "where a.name = $name "
              "return a.name as ndoe1, r.dist as dist, b.name as node2",
              {"name":str(i)}).to_data_frame()#dataframe에 graph를 저장하는 함수
    
    df_to_dict.append([])
    for j in range(len(df)):
        df_to_dict[i].append( (df.loc[j,'node2'],int(df.loc[j,'dist'])) )

# dataframe을 dictionary로 변환
# dictionary 구조 : 기준 노드 {가리키는 노드1:두 노드의 거리1, 가리키는 노드2:두 노드의 거리2, ...}
dic = dict()
for i in range(n):
    dic.update({str(i):dict(df_to_dict[i])})

print(dic)
print(type(dic))
print(len(dic))

{'0': {'3': 8, '2': 9, '1': 11}, '1': {'5': 8, '4': 8}, '2': {'6': 1, '3': 6, '1': 3}, '3': {'6': 10}, '4': {'5': 7}, '5': {'7': 5, '2': 12}, '6': {'7': 2}, '7': {'4': 4}}
<class 'dict'>
8


In [3]:
# Dijkstra 예제(3/4)
# 우선순위 큐 Dijkstra
# push를 이용해서 인접한 정점들 집어넣고 제일 작은애를 pop시키고
# 한개만 pop 두개는 그대로 남아있는데 
# 한개를 push하면 이전에 있던 노드들이랑 다시 계산해서 항상 최소값을 유지할 수 있는 구조라서 heap을 사용하는 것
def dijkstra(nodes,StartPoint,output):
    opt_road = {node:float('INF')for node in nodes}
    opt_road[StartPoint] = 0
    heap = []
    heapq.heappush(heap,[opt_road[StartPoint],StartPoint]) # 우선순위 큐
    
    while heap:
        distance, pos = heapq.heappop(heap)# 원소 꺼내기
        if opt_road[pos]<distance:
            continue
            
        for arrival, distance2 in nodes[pos].items():
            opt_dist = distance + distance2
            
            if opt_dist < opt_road[arrival]:
                opt_road[arrival] = opt_dist
                #경로저장
                output[int(arrival)]=[int(pos),int(arrival),opt_dist]
                heapq.heappush(heap,[opt_dist,arrival])# 원소삽입
    return opt_road, output

In [4]:
#dijkstra 알고리즘 실행 -0번 노드를 시작 노드로 지정

output = [[] for i in range(n)]
result, output = dijkstra(dic,'0',output)
print('result',result)
print('output',output)

result {'0': 0, '1': 11, '2': 9, '3': 8, '4': 16, '5': 19, '6': 10, '7': 12}
output [[], [0, 1, 11], [0, 2, 9], [0, 3, 8], [7, 4, 16], [1, 5, 19], [2, 6, 10], [6, 7, 12]]


In [5]:
#Dijkstra 예제(4/4)
# 그래프 생성 최종 결과
#연산된 거리 값을 각 노드에게 ㅐㅅ로운 property를 추가하여 입력
for i in range(n):
    nodes[i]['value']=result[str(i)]#node['new_property']:기존에 노드에 새로운 property 값 설정
    g.push(nodes[i]) #Graph.push(): 이미 생성된 노드에 변경된 부분만 바영
    
for i in output:
    if len(i)==0:
        output.remove(i)
        
road_nodes = [Node("OptimalNode",name=str(i)) for i in range(n)]

#transaction 실행
tx = g.begin()
for i in output:
    r = Relationship(road_nodes[i[0]],"OPTIMAL",road_nodes[i[1]],dist=str(i[2]))
    tx.create(r)
tx.commit()

if tx.finished()==False:
    print('transaction is still open')
else:
    print('transaction has been completed')

transaction has been completed


In [ ]:
#########  A* algorithm  #########
#best-First-Search
#각 정점이 매력함수값 g(x)를 갖고 있다.
# g(x)는 h(x)값 (heuristic 값 : 경험에 의해 얻어지는 방법)를 갖는다
# g(x)는 현재 노드에서 바로 다음노드까지의 거리 ( 8퍼즐에선 지금까지 움직인 횟수 )
# h(x)는 다음 노드에서 목표 노드까지의 추정 거리 ( 8퍼즐에선 제자리에 있지 않는 숫자의 갯수)
# 방문하지 않은 정점들 중 g(x) 값이 가장 매력적인 것부터 방문한다.
# >> Vertex x 로부터 목적점에 이르는 잔여거리의 추정치 h(x)는 실제치보다 크면 안된다.
# 합인 f(x)가 최소가 되는 방향으로 움직이면 목표에 도달하게 된다.
# h(x)==0 이 되는 노드가 목표노드

# 가중치는 g(x) 
# 추정 잔여거리 h(x)는 미리 계산 해놓는것
# 추정 잔여거리를 사용함으로써 탐색의 단계가 현저히 줄었다.

# 사용처 : 스타크래프트,8퍼즐,지도,,,
# 단, 목적지가 정해져있을때 유용함
# 추정치는 직선거리로 사용

In [1]:
f = open("file.txt",'w')
f.close

<function TextIOWrapper.close()>

In [2]:
f = open('file.txt','w')
for i in range(1,11):
    data = '%d번째 줄입니다.\n' % i
    f.write(data)
f.close()

In [5]:
f = open('file.txt','r')
line = f.readline()
print(line)
f.close()

1번째 줄입니다.



In [6]:
f = open('file.txt','r')
while True:
    line = f.readline()
    if not line: break
    print(line)
f.close()

1번째 줄입니다.

2번째 줄입니다.

3번째 줄입니다.

4번째 줄입니다.

5번째 줄입니다.

6번째 줄입니다.

7번째 줄입니다.

8번째 줄입니다.

9번째 줄입니다.

10번째 줄입니다.



In [8]:
f = open('file.txt','r')
lines = f.readlines()
print(lines)
for line in lines:
    print(line)
f.close()

['1번째 줄입니다.\n', '2번째 줄입니다.\n', '3번째 줄입니다.\n', '4번째 줄입니다.\n', '5번째 줄입니다.\n', '6번째 줄입니다.\n', '7번째 줄입니다.\n', '8번째 줄입니다.\n', '9번째 줄입니다.\n', '10번째 줄입니다.\n']
1번째 줄입니다.

2번째 줄입니다.

3번째 줄입니다.

4번째 줄입니다.

5번째 줄입니다.

6번째 줄입니다.

7번째 줄입니다.

8번째 줄입니다.

9번째 줄입니다.

10번째 줄입니다.



In [9]:
f = open('file.txt','r')
data = f.read()
print(data)
f.close()

1번째 줄입니다.
2번째 줄입니다.
3번째 줄입니다.
4번째 줄입니다.
5번째 줄입니다.
6번째 줄입니다.
7번째 줄입니다.
8번째 줄입니다.
9번째 줄입니다.
10번째 줄입니다.



In [6]:
print(type(a))

<class 'list'>


In [7]:
print(a)

[[0, 1, 11], [0, 2, 9], [0, 3, 8], [1, 4, 8], [1, 5, 8], [2, 1, 3], [2, 3, 6], [2, 6, 1], [3, 6, 10], [4, 5, 7], [5, 2, 12], [5, 7, 5], [6, 7, 2], [7, 4, 4]]
